In [25]:
from mongoengine import connect
import math
import json
import copy
import sys
sys.path.insert(0, '..\\mongo-db-scripts')
from Bug import Bug

connect('bug_reports_db', host='127.0.0.1', port=27017)

with open('data\\processed_verify_fields_final.json') as input:
  fields = json.load(input)

In [26]:
fields_presence_count = {}
current_fields_presence = {}

fields = [field['name'] for field in fields]
fields = list(filter(lambda field: field.startswith('cf_'), fields))

for field in fields:
  fields_presence_count[field] = 0
  current_fields_presence[field] = False

def verify_cfs_presence(history):
  current_presence = copy.deepcopy(current_fields_presence)

  for changes in history:
    for change in changes['changes']:
      field_name = change['field_name']

      if (field_name not in fields):
        continue
      
      current_presence[field_name] = True
  
  for field in fields_presence_count.keys():
    if current_presence[field]:
      fields_presence_count[field] += 1

In [ ]:
BUGS_LIMIT = 50000
total_bugs = Bug.objects.count()
bugs_per_iteration = math.ceil(total_bugs/BUGS_LIMIT)

bugs_counting_changes = []
temp_bug_init = {}

for field in fields:
  temp_bug_init[field] = 0

for i in range(bugs_per_iteration):
  skip_quantity = i * BUGS_LIMIT
  result = Bug.objects.only('bug_id', 'history').limit(BUGS_LIMIT).skip(skip_quantity)

  bugs = [bug.to_mongo().to_dict() for bug in result]

  for bug in bugs:
    verify_cfs_presence(bug['history'])
                    

In [29]:
with open('data\\processed_bugs_cfs_presence.json', 'w') as output:
  json.dump(fields_presence_count, output)